## Entraînement et évaluation

Nous allons passer maintenant à l'entrainement et l'évaluation du modèle. Nous utiliserons principalement la bibliothèque trainer de *tranformers*. Pour cela nous nécessiterons de :
- Définir un **collateur de données**. 
- une **métrique d'évaluation**
- partir d'un **modèle pré-entrainé**
- définir les **paramètres d'entrainement**


## Importation

Nous commençons par installer les librairies nécéssaires. Ici nous installons huggigface afin d'enregistrer et exporter notre modèle ré-entraîné sur hugging face.

In [ ]:
"""!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()"""

'!pip install huggingface_hub\nfrom huggingface_hub import notebook_login\n\nnotebook_login()'

On installe Git-LFS pour télécharger les points de contrôle de notre modèle :

In [ ]:
!pip install --upgrade pyyaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 33.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


Nous installons ensuite différents outils :
- *datasets* pour traiter les données en entrées
- *transformers* une brique du CTC
- *soundfile* pour traiter les audios
- *jiwer* pour calculer le taux d'erreur.

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install transformers==4.11.3
!pip install soundfile
!pip install jiwer

Tout d'abord nous avons besoin d'importer les extracteurs de descripteurs, tokenizers ainsi que la base de données.

In [ ]:
input_data_folder = "/content/data/data_40/"

In [ ]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained(input_data_folder+"Processor")  # Le processeur est une classe contenantle feature extractor ainsi que le tokenizer

from datasets import load_from_disk
timit = load_from_disk(input_data_folder +"dataset")
timit_prepared = load_from_disk(input_data_folder +"dataset_prepared")

timit

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 936
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 40
    })
})

### Collateur de données

Commençons par créer le collateur de données. Sans entrer dans les détails, à la différence des collateurs de données courants, ce collateur de données traite les valeurs d'entrée et les étiquettes différemment et leur applique donc des fonctions de remplissage distinctes (en utilisant à nouveau le gestionnaire de contexte de Wav2Vec2). Ceci est nécessaire car dans la parole, l'entrée et la sortie sont de modalités différentes, ce qui signifie qu'elles ne devraient pas être traitées par la même fonction de remplissage. De manière analogue aux collateurs de données communs, les padden tokens (tokens de remplissage) dans les étiquettes avec -100 de sorte que ces tokens ne sont pas pris en compte lors du calcul de la perte.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt"
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt"
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        print(labels)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Métrique d'évaluation

Courament, le Word Error Rate (WER) est utilisé mais dans notre cas (pour des plaques d'immatriculation) il est plus pertinent d'utiliser le Character Error Rate (CER). On peut aussi utiliser le Phoneme Error Rate (PER) qui est néanmoins beaucoup moins courant

In [ ]:
from datasets import load_metric
cer_metric = load_metric("cer", revision = "master")

In [ ]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

### Importation du modèle pré-entrainé

Maintenant, nous pouvons charger le modèle Wav2Vec2 pré-entraîné en français. Le `pad_token_id` du tokenizer doit être défini comme le `pad_token_id` du modèle ou, dans le cas de Wav2Vec2ForCTC, comme le token vide de CTC.

Pour économiser la mémoire du GPU, nous activons le point de contrôle du gradient de PyTorch et définissons également la réduction des pertes à "moyenne".

On peut trouver [ici](https://huggingface.co/models?sort=downloads&search=wav2vec) plusieurs modèles pré-entrainé de Wav2vec2.

In [ ]:
model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-french"


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    model_name , 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=59,
)


'''
attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
'''

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

'\nattention_dropout=0.1,\n    hidden_dropout=0.1,\n    feat_proj_dropout=0.0,\n    mask_time_prob=0.05,\n    layerdrop=0.1,\n'

Le premier composant de Wav2vec2 est une pile de couches de réseaux de neurones convolutifs (CNN) qui sont utilisés pour extraire des caractéristiques accoustiquement signifcatives (mais contextuellement indépendantes) du signal audio brut. Cette partie du modèle a été suffisamment entrainée durant le pré-entrainement and comme indiqué par les auteurs, ne nécessite pas d'être fine-tune. Donc nous pouvons définir le paramètre `required_grad` en `False` pour tous les paramètres de la partie d'extraction de caractéristiques.

In [ ]:
model.freeze_feature_extractor()

### Paramètres d'entrainement

Pour la partie finale, on définit tous les paramètres relatifs à l'entrainement. Pour donner plus d'informations : 

`group_by_length` rend l'entrainement plus effiface en regroupant en un même lot (batch), les échantillons de la base d'entrainement ayant une même taille d'entrée. Cela peut accélérer significativement le temps d'entrainement en réduisant énormément le nombre de padding tokens qui passent par le modèle.

`learning_rate` et `weight_decay` ont été heuristiquement régler jusqu'à que le fine-tuning soit stable. Notons que ces paramètres dépendent énormément de la base de données Timit et peut être non optimal pour d'autres base de données


In [ ]:
output_dir = "/content/data/data_40/model/" + model_name+"_HL"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=output_dir,
  push_to_hub=False,
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=100,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-3,
  weight_decay=0.000,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model.cuda(),
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 936
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2950


TypeError: ignored

In [ ]:
#trainer.push_to_hub() #Enregistrement sur huggingface

In [ ]:
#Enregistrement du processeur et modèle dans le répertoire
processor.save_pretrained(output_dir)
trainer.save_model(output_dir)

## Evaluation

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor, Wav2Vec2ForCTC
# Le processeur est une classe contenant le feature extractor ainsi que le tokenizer
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name).cuda()

"""
model_name = "Ilyes/wav2vec2-xls-fr"
model = Wav2Vec2ForCTC.from_pretrained(model_name) ## Modèle brut sans fine 
processor = Wav2Vec2Processor.from_pretrained(model_name)  
"""

In [ ]:
def map_to_result(batch):
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt", device="cuda"
  ).input_values

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
  return batch

In [ ]:
import torch
results_train = timit["train"].map(map_to_result)
results_test = timit["test"].map(map_to_result)

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=1):
    assert num_examples <= len(dataset), "On vérifie que le nombre d'exemples à afficher et inférieur au nombres de données"
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results_train["pred_str"], references=results_train["target_text"]))) # AFfiche score CER

df = pd.DataFrame(results_train.remove_columns(["speech","sampling_rate"]))
display(HTML(df.to_html())) # Affiche prédictions dans l'ordre

In [ ]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results_test["pred_str"], references=results_test["target_text"]))) # AFfiche score CER

df = pd.DataFrame(results_test.remove_columns(["speech","sampling_rate"]))
display(HTML(df.to_html())) # Affiche prédictions dans l'ordre